# Week 1: design and data representation
This week's tutorial/assignment will be about the basics of pattern analysis. More specifically, we'll discuss the difference in modelling/estimating patterns in within- and between-subject analyses, and subsequently how to load, transform, and represent these patterns for use in pattern analyses.

Additionally, we'll discuss some new programming concepts (mostly object-oriented programming), as this will become more and more important as you're going to learn how to program everything yourself in Python. 

## 1. Designs for pattern analyses
There are many ways in which you can categorize different types of pattern analyses (a topic which you'll work on in Thursdays's seminar), but one of the most basic categorizations is in terms of whether analyses are **within-subject** or **between-subject**. The major distinction revolves around whether you want to investigate an (experimental) factor that varies or is manipulated within subjects or that varies across subjects (i.e. individual differences or experimental between-subject designs). 

As explained in the lecture, there are three ways to extract patterns from fMRI data: 

1. extract activity at a certain timepoint (e.g. 3 TRs after stimulus presentation);
2. extract the *average* activity within a time-range (e.g. 2-4 TRs after stimulus presentation);
3. extract patterns of $\beta$-estimates by fitting a HRF per trial using the GLM you're familiar with; 

In the first part of this tutorial, you will see how the design matrices for the GLM in method 3 look like for both (single-trial) within-subject analyses and between-subject analyses. 

### 1.2. Within-subject designs
Often, trials in within-subject designs are modelled as separate regressors in a first-level analysis. In other words, you model each trial as a separate (*single!*) HRF-response. Below, we included an image of a single-trial design (of the hypothetical faces vs. houses experiment) as created in FSL:<img src="single_trial_design.png" alt="Drawing" heigth="100" width="400">

As you can see, each trial (either a face or a house) gets it's own regressor. Then, as depicted below the design matrix, a contrast-against-baseline is created for each regressor (trial), such that you can extract patterns of t-values (or z-values) for each trial.

Before you go on, make sure you understand this image! This image represents basically all you need to understand about single-trial designs. 

#### Implement your own single-trial design
In this section, you'll create your own single-trial matrix corresponding to a (real) working memory experiment (similar to the one described in Monday's lecture). In this experiment, one condition ("ACTIVE") subjects had to remember a configuration of bars and after a retention period had to respond whether one of the bars has changed in the test-image or not. In the other condition ("PASSIVE") they just watched a blank screen and had to respond with a random answer. The experiment is depicted schematically below:

![test](WM_example.png) 

In total, subjects performed 40 trials, of which 32 were of the "ACTIVE" condition and 8 were of the "PASSIVE" condition. Below, we'll load the onsets (and durations and conditions).

In [ ]:
# First some imports
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline

In [ ]:
# Numpy array with onsets (column 1), durations (column 2), and conditions (column 3)
# N.B.: condition 0 = passive, condition 1 = active
onsets = np.loadtxt('onsets.csv').astype(int)
print("Onset Duration Condition")
print(onsets)

<div class='alert alert-warning'>
**ToDo**: Practice your numpy-skills! How would you calculate how many active-trials and how many passive-trials there were? 
</div>

In [ ]:
# Implement your ToDo here


Remember, the onsets (and duration) are here defined in seconds (not TRs). Let's assume that the fMRI-run has a TR of 2. Now, we can convert (very easily!) the onsets/durations-in-seconds to onsets/durations-in-TRs.

<div class='alert alert-warning'>
**ToDo**: Convert the onsets and durations from seconds to TRs.
</div>

In [ ]:
# ToDo
onsets[:, :2] /= 2

Now, as you might remember, for each regressor we need to create a regressor of zeros and ones, in which the ones represent the moments in which the particular trial was presented. Remember, if a stimulus lasted 6 seconds (i.e. 3 TRs), make sure that your regressor also models your event for this duration! 

So, for example, if you have a (hypothetical) run with a duration of 15 TRs, and you show a stimulus at TR=3 for the duration of 3 TRs (i.e. 6 seconds), then you'd code your regressor as:

`[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]`

<div class='alert alert-warning'>
**ToDo**: Below, we initialized a stimulus vector (`stim_vec`) of shape=[40, 162], i.e. trials x timepoints (this run was 162 TRs long), with zeros. Each of the 40 rows represents one trial. Loop over the rows of the `stim_vec` matrix and fill the times at onset till the onset + 2 TRs with ones. Remember, the first index in Python is zero (not 1!).
</div>

In [ ]:
# Fill the stim_vec variable with ones at the indices of the onsets per trial!
stim_vec = np.zeros((162, 40))
for idx, onset in enumerate(onsets[:, 0]):
    stim_vec[range((onset -1), (onset + 1)), idx] = 1

Now, we only need to convolve an HRF with the stimulus-vectors and we'll have a complete single-trial design! Don't worry, we do this for you. We'll also plot it to see how it looks.

In [ ]:
from functions import double_gamma

hrf = double_gamma(range(162))

# List comprehension (fancy for-loop) + stack results back to a matrix
X = np.vstack([np.convolve(hrf, stim_vec[:, i], 'full')[:162] for i in range(40)]).T

plt.figure(figsize=(20, 10))
for plot in range(40):
    plt.subplot(1, 40, plot+1)
    plt.plot(X[:, plot], range(X.shape[0])[::-1])
    plt.axis('off')
    plt.text(-2, 170, 'Active' if onsets[plot, 2] else 'Passive', rotation=45)

plt.text(-60, -10, 'Regressors (trials)', ha='center', fontsize=20)
plt.text(-120, 80, 'Time (TR)', va='center', rotation='vertical', fontsize=20)
plt.show()

As explained in the lecture, we can use this design to extract patterns of $\beta$-values for each regressor. But [it has been shown](http://www.sciencedirect.com/science/article/pii/S1053811910007834) that converting $\beta$-values to t-values often creates more stable and robust patterns. As you might remember from "Neuroimaging: fMRI", $\beta$-values can be normalized to t-values by defining a contrast-against-baseline that is subsequently used in the formula for the t-value. 

Suppose I want to convert the patterns of $\beta$-values *for each trial* in the design above to t-values, how would my contrast-matrix look like? (Hint: check out the first image of this notebook.)

<div class='alert alert-warning'>
**ToDo**: create a design matrix in which each row represents the contrast-against-baseline-vector of a single trial of the within-subject design above. (Hint: check out the `numpy.eye()` function)
</div>

In [ ]:
# Do your ToDo here

### 1.3. Between-subject designs
Between-subject designs are way more 'simple' than within-subject designs. Basically, they're the same as traditional 'activation-based' (univariate) designs in which each condition (*not* trial!) gets its own regressor. Thus, the final design-matrix is, in this case of the WM experiment, only of size 162 x 2 (one 'active' regressor, one 'passive' regressor).

<div class='alert alert-warning'>
**ToDo**: Below, we again initialized a stimulus-vector matrix with zeros. Now fill it again with ones at times of the stimulus-onsets of each condition.   
</div>

In [ ]:
stim_vec_between = np.zeros((162, 2))
passive_idx = onsets[onsets[:, 2] == 0]
active_idx = onsets[onsets[:, 2] == 1]

stim_vec_between[passive_idx - 1, 0] = 1
stim_vec_between[passive_idx, 0] = 1
stim_vec_between[passive_idx + 1 , 0] = 1

stim_vec_between[active_idx -1 , 1] = 1
stim_vec_between[passive_idx, 1] = 1
stim_vec_between[passive_idx + 1 , 1] = 1

Now, given that you succesfully filled the stim_vec_between variable with ones, let's plot it again to see how this between-subject design looks like.

In [ ]:
Xb = np.vstack([np.convolve(hrf, stim_vec_between[:, i], 'full')[:162] for i in range(2)]).T

plt.figure(figsize=(2, 5))
for plot in range(2):
    plt.subplot(1, 2, plot+1)
    plt.plot(Xb[:, plot], range(Xb.shape[0])[::-1])
    plt.text(-2, 180, 'Passive' if plot == 0 else 'Active', rotation=45)
    plt.axis('off')
plt.text(-2, -15, 'Regressors (conditions)', ha='center', fontsize=10)
plt.text(-8, 80, 'Time (TR)', va='center', rotation='vertical', fontsize=10)
plt.show()

Hopefully, by creating design matrices for pattern extraction in within- and between-subject pattern analyses, you understand the differences between these two types better. 

In a real-life pattern analysis project, however, you'd probably just use FSL's FEAT to estimate the $\beta$/t-values of each trial/condition. We've done this for you: in the `part1`-folder (`/home/nim_?/week_1/part1`) you find two `.feat`-directories of a single subject - one as the result of a within-subject design and one as the result of a between-subject design.  

<div class='alert alert-info'>
**ToThink**: check out the files and folders in both directories. Look at both designs (double-click `design.png`) and check out the stats folders. Try to understand the following things: <br><br>

1. Why are there two `pe.nii.gz` (pe = parameter estimate) files for the BETWEEN.feat directory, but three cope/tstat/zstat files? (In other words, do you remember what how the pe/cope/varcope/zstat/tstat files relate to each other?)<br>
2. Why are there only 3 {cope/z/tstats} in the stats-directory of the \*BETWEEN.feat directory, and 40 in the \*WITHIN.feat directory?<br>

</div>

So far, we demonstrated the main differences between how you would define GLM design matrices to extract beta/t-value patterns for within- and between-subject pattern analyses. Now, we're going to digress slightly (but for a good reason) and talk about object-oriented programming in Python (something you'll encounter a lot in the coming weeks!).

## 2. Object-oriented programming
Python is a so-called `object-oriented` language. What this means is that *everything in Python is some kind of 'object'*, each having it's own *class*. Understanding these concepts is one of the most difficult things in Python (or any other object-oriented language), but it is vital to understand how the language works, and how to use it effectively. Also, in the coming weeks, you use packages (such as [scikit-learn](http://scikit-learn.org)) which strongly rely on Python's object-oriented functionality. 

In this section, we'll walk you through the basics of objects and classes in Python.

N.B.: if you already know about and understand the principles of object-oriented programming, skip this section by all means!

### 2.1.: classes vs. objects
As stated, everything in Python is an object, which is of a certain class. In fact, in the Python tutorial/refresher, you've learned about a couple of different classes already. For example, there is the `string` class:

In [ ]:
my_string = 'this is an object of the string-class'
print(type(my_string))

As you can see, you check the *class* of any object with the `type()` function. Let's check out some more types.

In [ ]:
obj1 = [1, 2, 3]
print(type(obj1))

obj2 = {'entry1': [1, 2, 3], 'entry2': 'testtesttest'}
print(type(obj2))

obj3 = 5
print(type(obj3))

At this moment, you might be thinking: what is the difference between objects and classes? Well, one way to think about it is to ***think about the relationship between an object and a class as the relationship between a building and it's (architectural) plan.***

In other words, just like a building plan outlines how a building should be constructed, does the class specify how an object should be created\*. Classes in Python define, basically, the attributes (*i.e. things that an object **is/has***) and the methods (*i.e. what each object **can do***).

\* In programmer-lingo, an *object is always an **instance** of a class*.

Remember (from the refresher) the distinction between methods and attributes? Let's look at (some of) the methods and attributes of a numpy-ndarray object:

In [ ]:
import numpy as np
my_array = np.array([1, 2, 3, 4])

print('The class of my_array is: %s\n' % type(my_array))

# Let's checkout some attributes
print(my_array.size)
print(my_array.shape)
print(my_array.ndim)

# And some methods ...
print(my_array.reshape((2, 2)))
print(my_array.prod())
print(my_array.mean())

As you can see, attributes (like `size`, `shape`, and `ndim`) are things that describe **things that an object has/is/characterizes**, and methods (like `reshape()`, `prod()`, and `mean()`) are **things that an object can do**. 

<div class='alert alert-warning'>
**ToDo**: Below, I imported some object with different types. Figure out the types. Which type has the particular method 'is_integer()'?
</div>

In [ ]:
# Check out the types; to which class belongs the method 'is_integer()'?
from functions import return_some_objects
obj1, obj2, obj3 = return_some_objects()

Alright, so let's reiterate what we know so far:

**Classes ("the plan") tell Python how to create objects ("the building"). A major 'responsibility' of classes is to specify which attributes and methods each object has.**

### 2.2: Custom classes
Now, Python (and other modules, like Numpy) have a bunch of built-in classes (like `str`, `list`, `dict`, etc.), but it's also possible to create your own classes! Below, we'll outline how we'd create a custom `Person` class, which has certain attributes and methods. Bear with us, we'll explain how each elements in the class-definition works step-by-step.

In [ ]:
class Person():
    """ This `person` class is a (nonsense) example to
    show how classes work. It takes the following parameters
    when initialized:
    
    name : a string
    age : an integer
    """
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def call_out_my_name(self):
        "This method calls out the name of the Person object. """
        print('My name is %s' % self.name)
        
    def update_age(self, number_to_add):
        """ This method updates the age attribute by a certain number.
        It takes one parameter, `number_to_add` (an integer). """
        self.age = self.age + number_to_add

Alright, so now we defined a custom `Person`-class. As you can see, it looks very similar to when you define a function (but instead of using "`def`" you use the keyword "`class`").

Remember, this is a "plan" (instruction) on how to create ("initialize") an object. Let's do this (i.e. initialize an object of class `Person`).

In [ ]:
my_person_object = Person(name='Lukas', age=25)

So, in the code-block above, we initialized a variable (or more specifically: the object) "my_person_object", which is an instance of the class `Person`. The way we initialized this object is how any object in Python is initialized\*:
![alt text](obj_class_diff.png)

\* "Huh, if any object is initialized this way (`obj = Class(inputs)`) then why don't I see that when I initialize builtin Python objects like `strings`, `lists`, `dictionaries`, etc.", you might righfully ask. Very good question, we'll get to that later!

Now, the `my_person_object` has the methods and attributes that have been specified by the `Person` class.

For example, as specified by the class, my_person_object now has the attributes `name` and `age`:

In [ ]:
print(my_person_object.name)

In [ ]:
print(my_person_object.age)

And it also has the methods `call_out_my_name()` and `update_age()`:

In [ ]:
my_person_object.call_out_my_name()

In [ ]:
number_to_update_age_with = 10
my_person_object.update_age(number_to_update_age_with)

<div class='alert alert-warning'>
**ToDo**: create a new instance of the `Person` class (e.g. with your own name and age) and mess around with it a little bit (e.g. call the update_age() function a couple of times and see how the age-attribute changes). 
</div>

In [ ]:
# Mess around with a new Person instance


So, alright, when we instantiated the my_person_object object according to the Person class, somehow the class "bound" the attributes and methods to the object. *How the hell does the class do that?*

#### 2.2.1: the __init__ method
If you look at the class definition of `Person` above, you might recognize functions (started by the keyword `def`) inside the class, like `__init__()` and `call_out_my_name()`. These are methods! Just remember: any `def` *inside* a class is a method (as explained in the refresher, methods are just functions applied to objects themselves). 

Let's talk about the `__init__()` method, a daunting but necessary element of each (custom) class. While it looks complicated (with all the underscores everywhere), it does just one simple thing: **it binds attributes to the object it*self***. 

Basically, when you initialize an object (e.g. by `Person(name='some name', age=42)`), Python will secretly call the `__init__()` method. By convention, the first argument to `__init__()` is *always* "self". What "self" represents, is a bit tricky to understand, but try to see it like this: "self" functions as a placeholder/template in the class, that will be 'filled in' by the specific object once it is initialized. 

Alright, complicated stuff. Let's try to clarify this statement by looking at another (simpler) example:

In [ ]:
class Computer():
    # os = operating system
    def __init__(self, os, owner):
        
        self.os = os
        self.owner = owner

In [ ]:
computer_object = Computer(os='Windows', owner='Noor')

Remember, a class is nothing more than an **instruction** ("plan") on how to create an object. As such, what you see in the Computer class definition, is an "instruction" that specifies that the attributes `os` and `owner` should be bound to the "placeholder" *self*. Once we *actually* initialize an object (as we did in the code cell above by `Computer(os='Windows', owner='Noor')`), the attributes ('Windows' and 'Noor') are not bound to the template *self* but to the actual `computer_object` object! That's why we can now access the `os` attribute from the `computer_object` itself:

In [ ]:
print(computer_object.os)

Thus, "self" only functions as a placeholder in the class (the "plan"/"instruction"), which is replaced by the actual object upon initialization.

Importantly, each argument that is listed in the `__init__()` method (here: `os` and `owner`) should be "filled in" when initializing a Computer object. Thus, for example, the following crashes because it expects and `owner` argument, but it hasn't received it during initialization:

In [ ]:
another_computer = Computer(os='Linux')

<div class='alert alert-info'>
**ToThink**: In the error message above, you see that `__init__()` is called when we tried to initialize a Computer-object, just as we expected! But the error also says that it takes 3 argument, but only received 2. This might seem weird, since we only passed *one* argument (namely `os`='Linux'), but this in fact makes sense. Why?
</div>

Alright, let's reiterate what we know of the `__init__()` method and its relation to classes so far:

1. It binds attributes to the object itself;
2. It always takes *self* as a first argument;
3. *self* functions as a "template" for when an object is actually initialized;
4. Any argument that is listed in the `__init__()` method will be expected upon initialization;



#### 2.1.2: other methods 
As discussed earlier, each function definition (recognizable by the `def` keyword) in a class refers to a method. Let's look at the `Person` class example again (copy-pasted from earlier in the document):

In [ ]:
class Person():
    """ This `person` class is a (nonsense) example to
    show how classes work. It takes the following parameters
    when constructed:
    
    name : a string
    age : an integer
    """
    
    def __init__(self, name, age):
        self.name = name
        self.age = age
    
    def call_out_my_name(self):
        "This method calls out the name of the Person object. """
        print('My name is %s' % self.name)
        
    def update_age(self, number_to_add):
        """ This method updates the age attribute by a certain number.
        It takes one parameter, `number_to_add` (an integer). """
        self.age = self.age + number_to_add

As we can see, apart from the `__init__()` method, there are two other methods: `call_out_my_name()` and `update_age()`. Like the `__init__()` method, they always expect *self* as the first argument. By always passing *self* to methods, we can access and use the attributes of the object. For example, the `call_out_my_name()` function accesses and uses the `name` attribute.

Apart from *self*, methods can also take additional "non-self" arguments. This you see for example in the `update_age()` method: as any method, it takes *self* as the first argument, but also needs an additional argument - `number_to_add`. 

Thus, methods can use "internal arguments" (the class attributes) and "external arguments" (which you have to provide when you call the method).

Then, inside the `update_age()` method, another interesting thing happens. We're actually modifying the `age` attribute. Whut?

Let's look at what that entails:

In [ ]:
personx = Person('Noor', 24) # Note that, like functions, you don't *need* to provide keywords, like name='Noor'
print("This person, named %s, is %i years old" % (personx.name, personx.age))

# Let's now update Noor's age (let's assume that it's 20-04, Noor's birthday)
personx.update_age(number_to_add=1)
print("This person is now %i years old!" % personx.age)

Note that because the `update_age()` method only modifies its own attributes, you don't need a `return` statement for the effect to take place. This construct of modifying internal attributes and thus omitting a return statement is also known as "in-place modifications". You can, however, both change (or even add) internal attributes *and* return things, like the following (more realistic) example shows:

In [ ]:
import nibabel as nib

class Niftiloader():
    # Tip: it is customary in Python to start custom classes with a capital letter!
    
    def __init__(self, path_to_nifti):
        # Here, we bind the argument path_to_nifti to self
        self.path_to_nifti = path_to_nifti
        
    def load_and_return_shape(self):
        
        nifti_file = nib.load(self.path_to_nifti)
        self.loaded_data = nifti_file.get_data()
        
        nifti_shape = nifti_file.shape
        
        return nifti_shape

The NiftiLoader class has a method `load_and_return_shape()` that both adds an attribute (namely `loaded_data`) *and* returns, additionally, the shape of the nifti-file (i.e. the voxel dimensions)! Let's first initialize a Niftiloader object and check which attributes it has after initialization.

In [ ]:
example_nifti = 'part1/sub-0037_workingmemory_WITHIN.feat/stats/tstat1.nii.gz'
my_loader = Niftiloader(path_to_nifti=example_nifti)

# path_to_nifti attribute exists!
print(my_loader.path_to_nifti)

# ... but loaded_data attributes doesn't (yet) ...
print(my_loader.loaded_data)

But if we call the `load()` function (and return the loaded data!), suddenly the `nifti_shape` attribute *does* exist!

In [ ]:
nif_shape = my_loader.load_and_return_shape()
print(my_loader.loaded_data)

# Also, we stored the shape that is returned from the load_and_return_shape() method into another variable ('nif_shape'),
print('\nThe shape of my nifti is: %s' % (nif_shape,))

<div class='alert alert-warning'>
**ToDo**: time to practice! We copy-pasted the `Niftiloader` class below, and we'd like you to extend the class as follows: add a method (named `calculate_min_and_max()`) that only takes *self* as an input argument and gets both the minimum value and maximum value from the `loaded_data` attribute. Then, it should create two new attributes (`min` and `max`, which should contain these values). Lastly, it should also return the extracted min and max value as a single tuple.
</div>

In [ ]:
import nibabel as nib

class Niftiloader():
    
    def __init__(self, path_to_nifti):
        
        self.path_to_nifti = path_to_nifti
        
    def load_and_return_shape(self):
        
        nifti_file = nib.load(self.path_to_nifti)
        self.loaded_data = nifti_file.get_data()
        
        nifti_shape = nifti_file.shape
        
        return nifti_shape
    
    # Add the new method here!
    def calculate_min_and_max(self):
        
        self.min = self.loaded_data.min()
        self.max = self.loaded_data.max()
        return(self.min, self.max)

### 2.3: classes in builtin Python objects
Remember that we told you that "everything in Python is an object"? And that each object is initialized according to a class, which takes the format: `obj = Someclass(args)`? You might have noticed that this format is not *always* used, most prominently in built-in Python classes, like `list`, `dict`, `string`, etc. 

Let's look at an example. We can initialize a list seemingly without a class:

In [ ]:
my_list_object = [1, 2, 3]
print(type(my_list_object))

In fact, what happened was that actually the `[]` functions as a "shortcut" for the class definition of a list! In fact, you can also just do:

In [ ]:
# just use the class definition list(input)
my_list_object = list([1, 2, 3])
my_list_object_with_brackets = [1, 2, 3]

# Is it really the same?
print(my_list_object == my_list_object_with_brackets)

So, using [] to initialize a list is just a shortcut for the class definition `list()`. In fact, this is true for many builtin Python objects. "Why?", you ask? Well, it's simply because programmers are lazy beings. Builtin types (like list, string, dict, etc.) are used so often that they invented (even) shorter "shortcuts" to initialize those objects. 

Other "shortcuts" for builtin types are:

In [ ]:
# To initialize a string
str('my string') # the same as 'my string'
dict(key1=[1, 2, 3], key2=[5, 2, 1]) # the same as {'key1': [1, 2, 3], 'key2': [5, 2, 1]}
tuple((1, 2, 3)) # the same as (1, 2, 3)

Alright, that should be enough to understand the object-oriented programming concepts we're going to encounter in the rest of this course! Now, we are going to work with some real (pattern-based) fMRI data and store it in a custom object!

But not after a short assignment!

<div class='alert alert-danger'>
**Assignment 1** (3 points): write a custom class, named `NiftiExplorer`, that is initialized with one parameter: `path_to_nifti` (referring to a string pointing to the path of a 4D nifti file). Then, write three methods: the first one - `load_data()` should read in the Nifti-file (using nibabel) and create a new attribute: `data` (which should be the loaded data as a 3D numpy array). The second method, named `demean_data()`, should subtract the temporal mean from each voxel in the `data` attribute and store the result in the `data` attribute again (effectively "updating" it). A third method, named `plot_histogram_of_values()`. This method should take the `data` attribute and make a histogram of all the values in using matplotlib (google how to do this if you forgot!).
</div>

In [ ]:
# This makes sure that you plot inside the notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# implement your class here!


In [ ]:
# if you want to test it, you can use this nifti-file
nifti_path = 'within_data/preproc/sub-0037/piopwm/func_data_mcst_sg_ss5.nii.gz'


## 3: Data representation
In pattern analyses, there is a specific way to 'store' and represent brain patterns: as 2D matrices of shape N-samples \* N-voxels. **Important**: often (and confusingly), people refer to voxels as (brain) 'features' in pattern analyses. So in articles people often refer to samples-by-features matrices!

Anyway, this is how such a matrix looks like: 
![](data_representation.png)

Each row thus represents the voxel pattern of a sample!

<div class='alert alert-info'>
**ToThink**: samples are defined differently for within- and between-subject designs, as you might remember from the lecture. What constitutes (typically) a sample in between-subject designs? And in within-subject designs?
</div>

As you can see, the originally 3D voxel patterns (e.g. whole-brain patterns of t-values) are flattened (also called "vectorized") such that we can stack all patterns vertically into a 2D matrix. There are two reasons why pattern analyses need this 2D format and thus discard spatial information about the voxel patterns:
 
1. There are very few analyses that take spatial information into account in the first place; 
2. Most algorithms used "under the hood" by pattern analyses rely heavily on matrix algebra (which operate on 2D matrices by definition).

Anyway, let's look at an example. We're going to work with the working-memory data (as outline in the beginning of the notebook). Suppose we want to investigate whether we can predict whether a trial is passive or active (factor: working memory load) from (whole-brain) voxel patterns. Consequently, this is a **within-subject design**. As such, we model each trial separately by fitting a single-trial design matrix to obtain patterns of t-values per trial (similar to the plot just before section **1.3**). The results are in the directory: `part1/sub-0037_workingmemory_WITHIN.feat`. Check out the directory (again) and especially the stats-folder. You should see a bunch of nifti-files which contain 3D voxel patterns with either "pe" (parameter estimates, FSLs way of referring to $\beta$s), "cope", "varcope", "zstat", or "tstat" values.

For this analysis, we're going to use patterns of t-stats (as is generally recommended over $\beta$s).<br>
As you can see, there are 40 nifti-files with t-stats; these refer to the 40 trials in the experiment (32 active trials, 8 passive trials)! Given that we need to adhere to the data representation as outlined above, we are in the following situation:

**What we have**: 40 (3D) nifti-files<br>
**What we need**: one 2D numpy array of shape 40 x {whatever amount of voxels there are in those niftis}

Alright, time to learn some Python gems that help us load in and transform those patterns into a usable 2D numpy matrix.

### 3.1: tips & tricks to load and transform (nifti-)files
As a first thing, we need to find all the paths to the tstat nifti-files. Python has a nifty (pun intended) tool called "`glob`" which can find files/directories on disk using [wildcards](https://en.wikipedia.org/wiki/Wildcard_character). It is usually imported as follows:

In [ ]:
from glob import glob

`glob`, in Python, is a function that takes a path (as a string) with one or more wildcard characters (such as the `*`) and searches for files/directories on disk that match that. For example, let's try to find all the png-images in the current directory using glob (these are the images that I used inside this notebook). 

In [ ]:
my_search_string = '*.png'
png_files = glob(my_search_string)
print(png_files)

As you can see, it returns a list with all the files/directories that matched the search-string. Note that you can also search files outside of the current directory (note that we're in the directory `/home/nipa_?/week_1` right now). To do so, we can simply specify the relative or absolute path to it. For example, suppose I would like to find all the home-directories of the students of this course. I could simply do:

In [ ]:
all_student_home_dirs = glob('/home/nipa_*')
print(all_student_home_dirs)

<div class='alert alert-warning'>
**ToDo**: Now you have the skills to actually "glob" all the t-stats yourself! Use glob to find all the paths to the t-stats and store the results (a list with 40 strings) in a variable called `tstat_paths`.
</div>

In [ ]:
# Implement the ToDo here!
tstat_paths = glob('part1/*WITHIN.feat/stats/tstat*.nii.gz')
tstat_paths = sorted(tstat_paths, key=lambda x: int(op.basename(x).split('.')[0].split('tstat')[-1]))
print(tstat_paths)

**Warning**: `glob` returns unsorted paths (so in seemingly random order). It's better if we sort the paths before loading them in, so the order of the paths is more intuitive (the first file is tstat1, the seconds tstat2, etc.). Python has a builtin function `sorted()`, which takes a list and sorts it alphabetically. The problem, here, is that if we'd use that - i.e. `sorted(tstat_paths)` - it will actually sort the files as: tstat1, tstat10, tstat11, etc. See for yourself: 

In [ ]:
print(sorted(tstat_paths))

To fix this issue, we wrote a little function (`sort_nifti_paths()`) that sorts the paths correctly. (If you're interested in how it works, check out the functions.py file in the week_1 directory.)

In [ ]:
# Let's fix it
from functions import sort_nifti_paths
tstat_paths = sort_nifti_paths(tstat_paths)

<div class='alert alert-warning'>
**ToDo**: in the code block below, write a loop that loads in the tstat nifti-files one by one (using nibabel) and store them in the already preallocated array "X". Note that "X" is a 2D matrix (samples-by-features), but each tstat-file contains a 3D array, so you need to "flatten" the 3D array to a single vector: use e.g. the numpy function "flatten()" or "ravel()". 
</div>

In [ ]:
voxel_dims = (80, 80, 37)
X = np.zeros((len(tstat_paths), np.prod(voxel_dims)))

# Start your loop here!

If you managed to load all the patterns of tstat-values in the "X" variable above, you have managed to do one of the most important things in pattern analyses: getting the data (patterns) in the right format: a 2D matrix of samples-by-features. From here onwards, we will learn how to apply pattern analyses to data in this format, which is (in a way) *much* easier because we will use mostly existing packages (like [scikit-learn](http://scikit-learn.org) for decoding analyses). 

For the final assignment of this week, you are going to create a custom class `Mvp` (which stands for **M**ulti**v**oxel **p**attern) that sort of "organizes" the loading process, data representation, and contains some methods that act upon (e.g. preprocess) the 2D samples-by-feature matrix. In the next weeks, you'll extend this Mvp-class with more functionality as you'll learn new things (e.g we'll add some decoding-related methods in week 2 and some RSA-related methods in week 3). 

<div class='alert alert-danger'>
**Assignment 2** (7 points). Create a new class, `Mvp`, that takes as input a list of paths to nifti-files that represent the samples in a given pattern analysis. This class should contain three methods:<br><br>
1. a method `load()` that loads the nifti-files (which are by now an attribute) and stores them in a 2D samples-by-features numpy array, which will be bound to self as a new attribute `X`;<br><br>
2. another method, `standardize()`, that standardizes each feature (voxel) in the 2D `data` attribute. Standardization is a common "preprocessing step" in pattern analyses that ensures that feature (columns in the 2D matrix) has mean 0 and standard deviation 1 (more info [here](https://en.wikipedia.org/wiki/Feature_scaling#Standardization)). Essentially, you just subtract each column's mean from each value in the column and then divide each value in the column by the standard deviation in the column;<br><br>
3. a method `apply_mask()`, that takes two parameters, `path_to_mask`, which should be a nifti-file containing a mask (e.g. a mask of the amygdala), and `threshold`, a number that should indicate the minimum value that a voxel in the mask should take to be included in the mask. This method should first load in the mask (using nibabel), then make a boolean array of the mask (array with True for voxels above the threshold, False for those below) and ravel (flatten) the result in a 1D vector, and finally use this boolean mask-array to index and update the columns of the `X` attribute such that afterwards it only contains voxels specified in the thresholded mask. <br><br>

**Hints**:<br><br>
1. Test your code! You can use the tstat_paths we "globbed" earlier from the `sub-0037_WITHIN.feat` directory; <br><br>
2. The methods do not need to return anything: everything is applied to the internal attributes (mostly the `X` attribute); <br><br>
3. We supplied a probabilistic amygdala mask for you (`/home/nipa_?/week_1/amygdala_mask.nii.gz`) to test your `apply_mask()` method with. If your method works correctly, your `X` attribute should be of shape [40, ...] after applying the mask to it.<br><br>
</div>